# Trending Topics: Assuming you receive Reddit comments as a live stream, build a Spark streaming application that will consume the stream and determine what topics are trending over a particular window of time.

In [47]:
from pyspark.streaming import StreamingContext
import json
# import yake

# The "1" here is the number of seconds between microbatches:
ssc = StreamingContext(spark.sparkContext, 1)

# Required to be able to do state updates:
ssc.checkpoint("checkpoint")

In [ ]:
# Updates our distribution
def update_dist(new_values, old_values):
    return sum(new_values) + (old_values or 0)

In [43]:
import pandas as pd
import matplotlib.pyplot as plt

# Assumes the stream is running on the same machine as the driver.
# ./streamer2.sh 23999 /bigdata/mmalensek/data/reddit/2005/RC_2008-12.bz2

sock = ssc.socketTextStream("orion05", 23999)

# Updates our distribution
def update_dist(new_values, old_values):
    return sum(new_values) + (old_values or 0)

def map_get_body(s):
    '''
    The purpose of this function is to get the content of the body from a string
    '''
    # initializing substrings
    sub1 = '"body":"'
    sub2 = '","'

    s=str(re.escape(sub1))
    e=str(re.escape(sub2))

    # printing result
    res=re.findall(s+"(.*)"+e,test_str)[0]
    
    return res

def keywords_extractor(s):
    '''
    Using yake library, this function will return an important keyword on a body
    '''
    kw_extractor = yake.KeywordExtractor()
    language = "en"
    max_ngram_size = 3
    deduplication_threshold = 0.9
    numOfKeywords = 1
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    
    return (keywords, 1)

# Plots the distribution. This executes on the driver!
def show_df(rdd):
    '''
    This function purpose is to show top 5 trending topic
    '''
    df = pd.DataFrame(rdd.collect(), columns=['trending_topic', 'frequency'])
    df.sort_values('trending_topic', ascending=False)
    df.head(5)

body = sock.map(map_get_body)\
        .map(keywords_extractor)\
        .reduceByKey(lambda x, y: x + y)

distrib = reduced.updateStateByKey(show_df)

# Plot the new distribution for each microbatch
distrib.foreachRDD(plot_distribution)

body.pprint()

In [44]:
# Running this will start listening:
ssc.start()

In [45]:
# IMPORTANT: you need the stopSparkContext=False, otherwise
# your driver will die and you'll have to restart Jupyter
ssc.stop(stopSparkContext=False)

In [1]:
sc = spark._jsc.sc() 
n_workers =  len([executor.host() for executor in sc.statusTracker().getExecutorInfos() ]) -1

print(n_workers)

10
